### Analyse data USGS - Earthquake 1965 - 2016

In [1]:
# setup absolute path to location of package Starts and config-file 
from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])

# import data in Pandas DataFrame
from Starts.startml import *
from Starts.startspk import *

local_kwargs {'data_path': './data/earthquake-database.csv', 'drop_obj_col': False, 'nan_drop_col': False, 'nan_drop_row': False, 'nan_zero': False, 'nan_mean': True, 'nan_mean_neighbors': False}


In [2]:
train_data=idata[0]
train_data.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [3]:
# create data with features: Time, Latitude, longitude, depth, magnitude, location source
data = train_data[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude', 'Location Source']]

# export data in csv-format without header
data.to_csv('./data/earthquake.csv', header=False)

# import data using Spark
spk = StartSPK(app_name="usgs", path_files=["./data/earthquake.csv"], rdd=True)
dat, sc, ss = spk.get_dat()
dat

[./data/earthquake.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0]

In [7]:
type(dat)

list

In [10]:
dat=dat[0]

In [11]:
# show the first 10 elements in RDD dat.collect() to show all
dat.take(10)

['0,01/02/1965,13:44:18,19.246,145.616,131.6,6.0,ISCGEM',
 '1,01/04/1965,11:29:49,1.8630000000000002,127.352,80.0,5.8,ISCGEM',
 '2,01/05/1965,18:05:58,-20.579,-173.972,20.0,6.2,ISCGEM',
 '3,01/08/1965,18:49:43,-59.076,-23.557,15.0,5.8,ISCGEM',
 '4,01/09/1965,13:32:50,11.937999999999999,126.427,15.0,5.8,ISCGEM',
 '5,01/10/1965,13:36:32,-13.405,166.62900000000002,35.0,6.7,ISCGEM',
 '6,01/12/1965,13:32:25,27.357,87.867,20.0,5.9,ISCGEM',
 '7,01/15/1965,23:17:42,-13.309000000000001,166.21200000000002,35.0,6.0,ISCGEM',
 '8,01/16/1965,11:32:37,-56.452,-27.043000000000003,95.0,6.0,ISCGEM',
 '9,01/17/1965,10:43:17,-24.563000000000002,178.487,565.0,5.8,ISCGEM']

#### Tasks: RDD & pairRDD 
#### given location e.g. US, find/ count how many (data points) rows which happened at this location

In [12]:
def parseLine(line):
    fields = line.split(',')
    idx = int(fields[0])
    date = str(fields[1])
    time = str(fields[2])
    lat = float(fields[3])
    lon = float(fields[4])
    dep = float(fields[5])
    mag = float(fields[6])
    loc = str(fields[7])
    return (loc, date, time, lat, lon, dep, mag)

rdd = dat.map(parseLine)
rdd.take(5)

[('ISCGEM', '01/02/1965', '13:44:18', 19.246, 145.616, 131.6, 6.0),
 ('ISCGEM', '01/04/1965', '11:29:49', 1.8630000000000002, 127.352, 80.0, 5.8),
 ('ISCGEM', '01/05/1965', '18:05:58', -20.579, -173.972, 20.0, 6.2),
 ('ISCGEM', '01/08/1965', '18:49:43', -59.076, -23.557, 15.0, 5.8),
 ('ISCGEM', '01/09/1965', '13:32:50', 11.937999999999999, 126.427, 15.0, 5.8)]

In [13]:
rdd.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.colle

In [14]:
# filter location 'US' out and count how many locations earthquake happened
usrdd = rdd.filter(lambda r: 'US' in r[0])
othersrdd = rdd.filter(lambda r: 'US' not in r[0])
usrdd.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.colle

In [ ]:
# other locations
othersrdd.count()

In [ ]:
usrdd.take(5)

In [ ]:
usrdd.take(1)[0][5]

In [ ]:
# Location in US with max and min magnitude 
usrdd.max(lambda x: x[6]), usrdd.min(lambda x: x[6])

In [ ]:
# Location in US with max and min depth 
usrdd.max(lambda x: x[5]), usrdd.min(lambda x: x[5])

In [ ]:
type(usrdd)

In [ ]:
usrdd.take(2)

#### Tasks: SparkSQL

In [ ]:
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType
# from pyspark.sql.types import DateType
# from pyspark.sql.types import TimestampType
from pyspark.sql.types import DoubleType

# create schema for field
schema = StructType([StructField("Country", StringType(), True), 
                     StructField("Date", StringType(), True), 
                     StructField("Time", StringType(), True), 
                     StructField("Latitude", DoubleType(), True),
                     StructField("Longitude", DoubleType(), True), 
                     StructField("Depth", DoubleType(), True),
                     StructField("Magnitude", DoubleType(), True)
                     ])
schema

In [ ]:
# convert rdd object into DataFrame object and execute SQL operations on DataFrame object
df = ss.createDataFrame(usrdd, schema).cache()
df

In [ ]:
type(df)

In [ ]:
df.take(1)

In [ ]:
# register the DataFrame as a table for SQL-query
df.createOrReplaceTempView("EarthQuake")

In [ ]:
# query data with condition via SQL-command
loc_mags = ss.sql("SELECT * FROM EarthQuake WHERE Magnitude >= 9")
for loc in loc_mags.collect():
    print(loc)

In [ ]:
loc_depth = ss.sql("SELECT * FROM EarthQuake WHERE Magnitude >=7 and Depth>=650")
loc_depth.count()

In [ ]:
loc_depth.collect()